# 1) Imports

In [1]:
import pandas as pd
import geopandas as gpd
import warnings
import unidecode
import re

from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from geopy.distance import great_circle
from shapely import wkt
from shapely.geometry import shape

warnings.filterwarnings('ignore')

# 2) Coleta dos Dados

## 2.1) Lista de municípios do Sertão

In [2]:
def limpar_nome(nome):
    nome_sem_acentos = unidecode.unidecode(nome)  
    nome_sem_codigo_estado = re.sub(r'\(\w+\)', '', nome_sem_acentos)  
    return nome_sem_codigo_estado.upper()  

mun_ser = pd.read_excel(r'../Dados/Municipios/lista-1262municipios-semiarido-2017.xlsx', header = 2).iloc[1:]
mun_ser = mun_ser[['Cod IBGE','UF', 'Município']]
mun_ser.columns = ['IBGE7', 'UF', 'NOME']

mun_cord = pd.read_csv(r'../Dados/Municipios/municipios.csv')
mun_cord = mun_cord[['codigo_ibge', 'latitude', 'longitude']]
mun_cord.columns = ['IBGE7', 'LATITUDE', 'LONGITUDE']

mun_ser = mun_ser.merge(mun_cord, on ='IBGE7', how = 'left')

mun_ser['NOME'] = mun_ser['NOME'].apply(limpar_nome)

mun_ser.to_csv('../Dados/Views/municipios_sertao.csv')
mun_ser

,IBGE7,UF,NOME,LATITUDE,LONGITUDE
0,2700300,AL,ARAPIRACA,-9.75487,-36.6615
1,2700706,AL,BATALHA,-9.67420,-37.1330
2,2700904,AL,BELO MONTE,-9.82272,-37.2770
3,2701209,AL,CACIMBINHAS,-9.40121,-36.9911
4,2701605,AL,CANAPI,-9.11932,-37.5967
...,...,...,...,...,...
1256,2806008,SE,RIBEIROPOLIS,-10.53570,-37.4380
1257,2807006,SE,SAO MIGUEL DO ALEIXO,-10.38470,-37.3836
1258,2807105,SE,SIMAO DIAS,-10.73870,-37.8097
1259,2807303,SE,TELHA,-10.20640,-36.8818


## 2.2) Dados de registros administrativos 

In [3]:
def calcular_populacao_total(municipio, municipios_df):
    populacao_total = municipio['POP_TOT']  
    for _, row in municipios_df.iterrows():
        distancia = great_circle((municipio['LATITUDE'], municipio['LONGITUDE']), (row['LATITUDE'], row['LONGITUDE'])).kilometers
        if distancia <= 30:
            populacao_total += row['POP_TOT']
    return populacao_total

imputer = KNNImputer(n_neighbors=5, weights="uniform")
min_max = MinMaxScaler()

metrics = [
    'ANO', 
    'IBGE7',
    'DIST_EF_PUB',
    'DIST_EM_PUB',
    'IDEB_AI',
    'IDEB_AF',
    'DOCSUP_EF_PUB',
    'DOCSUP_EM_PUB',
    'TXNASC7C',
    'TXNBAIXOP',
    'PINTERSAP',
    'PINTERDRSAI',
    'PDEFAGUA',
    'PDEFESGOTO',
    'PIND_POS',
    'SNIS_PAGUA',
    'POP_TOT', 
    "PMATPUB_EF", 
    "PMATPUB_EM"
]

reg_adm_mun = pd.read_excel(
    r'../Dados/Registros Administrativos/dados_registros_administrativos_total_2012_2017.xlsx', 
    sheet_name = 'MUNICÍPIO'
)

reg_adm_mun = reg_adm_mun[reg_adm_mun['IBGE7'].isin(mun_ser['IBGE7'])]
reg_adm_mun = reg_adm_mun[metrics]
num_metrics = [metric for metric in reg_adm_mun.columns if metric !='IBGE7']

reg_adm_mun[num_metrics] = min_max.fit_transform(reg_adm_mun[num_metrics])
reg_adm_mun[num_metrics] = imputer.fit_transform(reg_adm_mun[num_metrics])
reg_adm_mun[num_metrics] = min_max.inverse_transform(reg_adm_mun[num_metrics])

reg_adm_mun = reg_adm_mun.drop(columns='ANO')
reg_adm_mun = reg_adm_mun.groupby(['IBGE7']).mean().reset_index()

cidades = mun_ser.copy()
cidades = cidades.merge(reg_adm_mun, on = 'IBGE7', how = 'inner')
cidades['POP_TOT_30KM'] = 0
for index, municipio in cidades.iterrows():
    populacao_total_30km = calcular_populacao_total(municipio, cidades)
    cidades.at[index, 'POP_TOT_30KM'] = populacao_total_30km

cidades = cidades.drop(columns = ['UF', 'NOME', 'LATITUDE', 'LONGITUDE'])
cidades.to_csv('../Dados/Views/registros_administrativos.csv')
cidades

,IBGE7,DIST_EF_PUB,DIST_EM_PUB,IDEB_AI,IDEB_AF,DOCSUP_EF_PUB,DOCSUP_EM_PUB,TXNASC7C,TXNBAIXOP,PINTERSAP,PINTERDRSAI,PDEFAGUA,PDEFESGOTO,PIND_POS,SNIS_PAGUA,POP_TOT,PMATPUB_EF,PMATPUB_EM,POP_TOT_30KM
0,2700300,28.884,43.100,4.580,3.624,54.40,56.96,60.158262,10.090574,18.636148,6.815611,24.4256,78.4716,38.7872,97.486,230975.6,74.732,77.548,582087.0
1,2700706,31.100,35.892,4.544,3.632,12.62,41.94,55.029615,7.258357,30.860307,12.150951,44.1148,83.7320,66.1828,88.194,18487.6,89.920,97.592,120483.6
2,2700904,36.412,54.792,3.928,3.044,27.12,31.84,53.794486,8.063568,20.308137,6.723170,58.8868,83.3944,39.4776,91.050,6774.4,100.000,100.000,114803.4
3,2701209,38.284,40.720,4.164,3.324,49.04,45.22,52.443551,6.698345,17.231791,2.046380,75.6284,65.0140,51.4104,32.604,10815.6,91.742,99.436,141797.4
4,2701605,31.024,38.636,4.584,3.580,20.84,32.34,52.780364,8.178384,12.902533,1.653252,72.2388,56.8692,37.9112,94.054,17958.4,100.000,100.000,148162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,2806008,35.080,41.796,4.032,3.472,64.68,77.28,73.853258,6.865284,8.830588,0.186191,24.5524,41.4504,47.4972,96.058,18359.0,80.506,100.000,100748.8
1257,2807006,33.148,47.912,3.940,3.116,62.08,66.38,55.382784,8.533114,16.568992,1.666673,33.4896,48.7640,51.4924,95.810,3901.6,100.000,100.000,127390.8
1258,2807105,38.340,45.756,4.200,3.364,61.38,82.96,54.129258,7.443652,20.306567,0.782715,45.4120,42.6500,61.3536,97.866,40522.2,85.434,89.606,120552.4
1259,2807303,31.636,47.600,4.384,3.520,45.66,71.74,59.483911,7.337448,12.372443,0.612512,19.4664,24.3776,61.3848,98.560,3169.6,100.000,100.000,81132.8


## 2.3) Dados meteorológicos 

In [4]:
def calcular_distancia(cidade, estacao):
    cidade_coord = (cidade['LATITUDE'], cidade['LONGITUDE'])
    estacao_coord = (estacao['LATITUDE'], estacao['LONGITUDE'])
    return great_circle(cidade_coord, estacao_coord).kilometers

imputer = IterativeImputer(random_state=42)

ufs = list(set(mun_ser['UF']))
est_met = pd.DataFrame()

num_columns = [
    'LATITUDE', 
    'LONGITUDE',
    'ALTITUDE',
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",
    "RADIACAO GLOBAL (Kj/m²)",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"
]

for year in [2020, 2021, 2022]:
    
    file_path = f'../Dados/INMET/INMET_{year}.csv'
    file = pd.read_csv(file_path)
    file = file[
        [
            'Data', 
            'UF', 
            'ESTACAO', 
            'LATITUDE', 
            'LONGITUDE',
            'ALTITUDE',
            "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",
            "RADIACAO GLOBAL (Kj/m²)",
            "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"
        ]
    ]
    file = file[file['UF'].isin(ufs)]
    file = file.dropna(axis = 0, thresh = 8)
    
    # Imputation step:
    file[num_columns] = imputer.fit_transform(file[num_columns])
    

    file = file.groupby(['Data', 'UF', 'ESTACAO','LATITUDE', 'LONGITUDE','ALTITUDE']).agg(
        {
            "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": 'sum', 
            "RADIACAO GLOBAL (Kj/m²)":'mean',
            "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": 'mean'
            
        }
    ).reset_index()
    est_met = pd.concat([est_met, file], ignore_index=True)

est_met = est_met.drop(columns = 'Data')
est_met = est_met.groupby(['ESTACAO', 'UF']).mean().reset_index()
est_met.columns = [
    'ESTACAO', 
    'UF', 
    'LATITUDE', 
    'LONGITUDE', 
    'ALTIUDE',
    'PREC_MED', 
    'RED_MED',
    'TEMP_MED'
]

cidades = mun_ser.copy()

for index_cidade, cidade in cidades.iterrows():
    distancias = []
    for index_estacao, estacao in est_met.iterrows():
        distancia = calcular_distancia(cidade, estacao)
        distancias.append(distancia)
    indice_estacao_mais_proxima = distancias.index(min(distancias))
    estacao_mais_proxima = est_met.loc[indice_estacao_mais_proxima]
    cidades.at[index_cidade, 'PREC_MED'] = estacao_mais_proxima['PREC_MED']
    cidades.at[index_cidade, 'RED_MED'] = estacao_mais_proxima['RED_MED']
    cidades.at[index_cidade, 'TEMP_MED'] = estacao_mais_proxima['TEMP_MED']
    
cidades['PREC_MED'] = cidades['PREC_MED']*365
cidades = cidades.drop(columns = ['UF', 'NOME', 'LATITUDE', 'LONGITUDE'])
cidades["PREC_MED"] = cidades["PREC_MED"]*365 
cidades.to_csv('../Dados/Views/dados_meteorologicos.csv')
cidades

IBGE7     PREC_MED      RED_MED   TEMP_MED   VEL_MED
0     2700300   988.689803  1051.169614  26.157050  1.553386
1     2700706   713.023256  1318.900553  27.754333  1.427155
2     2700904   713.023256  1318.900553  27.754333  1.427155
3     2701209   209.280265  1585.450518  26.482644  1.062182
4     2701605   551.553841  1437.777193  27.047011  2.518969
...       ...          ...          ...        ...       ...
1256  2806008   362.694737  1146.244737  27.008460  3.015386
1257  2807006   388.322078  1739.639845  27.404957  2.741722
1258  2807105   162.378722  1370.066485  26.034025  0.710818
1259  2807303  1517.640390  1621.663140  26.874591  1.169412
1260  2807402   920.296117  1180.190480  25.148068  2.615223

[1261 rows x 4 columns]

## 2.4) Dados de recursos hídricos

In [5]:
atl_irr = pd.read_excel(r'../Dados/ANA/Atlas Irrigacao - Area Atual e Potencial.xlsx',header = 6).iloc[6:]
atl_irr = atl_irr[['Código','Área Total Irrigada', 'AAI - Potencial Total', 'AAI - Potencial Efetivo']]
atl_irr = atl_irr[atl_irr['Código'].isin(mun_ser['IBGE7'])]
atl_irr.columns = ['IBGE7','AREA_IRRIGADA_TOT', 'AREA_IRRIGADA_POT', 'AREA_IRRIGADA_POT_E']
atl_irr['AREA_IRRIGADA_TOT_POT_E'] = atl_irr['AREA_IRRIGADA_TOT'] + atl_irr['AREA_IRRIGADA_POT_E']
atl_irr.to_csv('../Dados/Views/recursos_hidricos.csv')
atl_irr

,IBGE7,AREA_IRRIGADA_TOT,AREA_IRRIGADA_POT,AREA_IRRIGADA_POT_E,AREA_IRRIGADA_TOT_POT_E
465,2100907,56.000000,829.401141,3.140248,59.140248
653,2112209,666.262426,4269.334204,0.000000,666.262426
667,2200053,96.000000,0.000000,0.000000,96.000000
670,2200251,0.000000,1240.907963,0.000000,0.000000
671,2200277,2.000000,0.000000,0.000000,2.000000
...,...,...,...,...,...
3076,3170651,353.204987,1106.772079,142.399238,495.604224
3079,3170800,4133.645939,37599.860831,1881.799953,6015.445892
3080,3170909,197.000000,400.497303,1.268556,198.268556
3082,3171030,4822.626564,295.487156,3.395124,4826.021688


## 2.5) Dados do censo

In [6]:
censo = pd.read_excel(r'../Dados/Censo/Censo_municipal_estadual_nacional.xlsx', sheet_name = 'MUN 91-00-10')
censo = censo[(censo['Codmun7'].isin(mun_ser['IBGE7']))&(censo['ANO']==2010)]
censo_colunas = [
    'Codmun7', 
    'IDHM', 
    'T_BANAGUA',
    'T_DENS',
    'T_LIXO',
    'T_LUZ',
    'AGUA_ESGOTO',
    'PAREDE', 
    'PIND', 
    'PINDCRI', 
    'T_DES18M', 
    'I_FREQ_PROP', 
    'T_FBPRE', 
    'T_FBSUPER', 
    'T_ATRASO_1_BASICO', 
    'T_ATRASO_1_FUND', 
    'T_ATRASO_1_MED',
    'T_ATRASO_2_BASICO', 
    'T_ATRASO_2_FUND', 
    'T_ATRASO_2_MED', 
    'T_ANALF11A14',
    'T_ANALF15A17', 
    'T_ANALF15M', 
    'T_ANALF18A24', 
    'T_ANALF18M', 
    'T_ANALF25A29',
    'T_ANALF25M',
    'RDPC'
]

censo = censo[censo_colunas]
censo.columns = censo_colunas
censo.rename(columns={'Codmun7':'IBGE7'}, inplace=True)
censo.to_csv('../Dados/Views/censo.csv')
censo

,IBGE7,IDHM,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,PIND,PINDCRI,...,T_ATRASO_2_FUND,T_ATRASO_2_MED,T_ANALF11A14,T_ANALF15A17,T_ANALF15M,T_ANALF18A24,T_ANALF18M,T_ANALF25A29,T_ANALF25M,RDPC
11594,2100907,0.521,31.94,42.14,71.79,95.54,23.23,33.06,40.55,49.69,...,25.21,12.84,11.16,9.96,33.70,13.71,36.20,22.16,42.60,175.81
11782,2112209,0.649,76.38,39.60,77.55,98.66,7.39,17.06,9.62,15.38,...,19.87,10.84,6.32,3.28,17.26,4.16,18.49,7.86,22.40,365.26
11796,2200053,0.528,19.28,36.52,99.22,98.99,23.32,0.53,32.12,46.41,...,20.35,15.65,7.49,4.62,30.41,6.79,32.67,14.49,39.08,199.35
11799,2200251,0.531,48.55,24.04,97.24,86.10,10.77,0.21,28.91,41.95,...,35.54,6.81,15.70,11.72,44.40,14.57,46.90,27.83,53.81,222.87
11800,2200277,0.585,72.12,34.60,97.43,98.34,3.89,0.00,22.97,32.09,...,26.90,3.28,4.73,5.70,35.26,6.92,37.95,17.77,45.68,268.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14205,3170651,0.634,75.25,18.35,88.58,96.62,16.25,0.16,15.35,21.42,...,9.77,3.17,2.81,1.82,25.27,3.03,27.77,8.74,34.48,265.02
14208,3170800,0.666,85.76,26.52,97.24,99.13,9.12,0.63,4.81,6.75,...,17.76,9.39,1.85,1.15,12.00,1.94,12.97,3.02,15.69,413.59
14209,3170909,0.594,66.33,37.32,67.62,98.34,17.92,0.38,22.81,30.83,...,20.02,18.77,2.58,2.42,24.45,4.64,27.06,8.40,33.46,238.92
14211,3171030,0.584,59.85,40.46,95.64,95.35,16.28,4.68,14.73,18.72,...,22.74,9.13,6.76,4.15,26.85,6.22,30.03,12.69,37.53,233.27


## 2.6) Qualidade da água

In [7]:
imputer = KNNImputer(n_neighbors=5, weights="uniform")
min_max = MinMaxScaler()

ql_agua = pd.read_csv(r'../Dados/ANA/Indicadores de Qualidade da Agua.csv')
ql_agua = ql_agua[['CDESTACAO','CORPODAGUA', 'SGUF','LATITUDE', 'LONGITUDE', 'MED_2020','MED_2021']]
ql_agua = ql_agua[ql_agua['SGUF'].isin(ufs)]
ql_agua = ql_agua.dropna(axis = 0, thresh = 3)

num_columns = ['LATITUDE', 'LONGITUDE', 'MED_2020','MED_2021']

ql_agua[num_columns] = min_max.fit_transform(ql_agua[num_columns])
ql_agua[num_columns] = imputer.fit_transform(ql_agua[num_columns])
ql_agua[num_columns] = min_max.inverse_transform(ql_agua[num_columns])

ql_agua['QUAL_MED_AGUA'] = (ql_agua['MED_2021'] + ql_agua['MED_2020'])/2
ql_agua = ql_agua.reset_index().drop(columns='index')

cidades = mun_ser.copy()
dist_corpo_agua_mais_prox = []

for index_cidade, cidade in cidades.iterrows():
    distancias = []
    for index_estacao, estacao in ql_agua.iterrows():
        distancia = calcular_distancia(cidade, estacao)
        distancias.append(distancia)
    indice_estacao_mais_proxima = distancias.index(min(distancias))
    dist_corpo_agua_mais_prox.append(min(distancias))
    estacao_mais_proxima = ql_agua.loc[indice_estacao_mais_proxima]
    cidades.at[index_cidade, 'QUAL_MED_AGUA'] = estacao_mais_proxima['QUAL_MED_AGUA']
#cidades['DIST_CORPO_AGUA'] = dist_corpo_agua_mais_prox
cidades = cidades.drop(columns = ['UF', 'NOME', 'LATITUDE', 'LONGITUDE'])
cidades.to_csv('../Dados/Views/qualidade_da_agua.csv')
cidades

,IBGE7,QUAL_MED_AGUA
0,2700300,62.268456
1,2700706,66.758485
2,2700904,66.758485
3,2701209,66.758485
4,2701605,71.338777
...,...,...
1256,2806008,63.883423
1257,2807006,63.883423
1258,2807105,63.883423
1259,2807303,66.862432


## 2.7) Dados de transporte

In [8]:
trans = pd.read_excel('../Dados/Transportes/transport_cost.xlsx')
trans = trans[['codigo_ibge','transportation_cost']].groupby('codigo_ibge').min().reset_index()
trans.columns = ['IBGE7', 'TRANSPORT_COST']
trans.to_csv('../Dados/Views/custo_de_transporte.csv')
trans

,IBGE7,TRANSPORT_COST
0,1400100,40478.947612
1,1400282,40326.183579
2,1501402,33091.358691
3,1501600,32666.086719
4,1504802,35154.250138
...,...,...
1380,5213806,25385.396775
1381,5214051,27720.752548
1382,5217203,27268.044018
1383,5219803,26161.653604


## 2.8) Variáveis educacionais

In [9]:
imputer = KNNImputer(n_neighbors=5, weights="uniform")
min_max = MinMaxScaler()

edu_raw_2020 = pd.read_csv(r"..\Dados\Q EDU\Dados_QEdu_Analitico_2020.csv", delimiter=";")
edu_2020 = edu_raw_2020.copy()


codigos_ne = list(range(21, 30))
edu_2020_nordeste = edu_2020[edu_2020["COD UF"].isin(codigos_ne)]

edu_2020_nordeste["Total de escolas estaduais"] = edu_2020_nordeste["Total de escolas estaduais"].str.replace(",", ".")
edu_2020_nordeste["Total de escolas municipais"] = edu_2020_nordeste["Total de escolas municipais"].str.replace(",", ".")

edu_2020_nordeste[[ 
                "Total de escolas estaduais", 
                "Total de escolas municipais"]] = edu_2020_nordeste[[ 
                                                                    "Total de escolas estaduais", 
                                                                    "Total de escolas municipais"]].astype('float').astype('Int64')

colunas_edu = ["Total de escolas estaduais", "Total de escolas municipais"]

# Imputation step:
edu_2020_nordeste[colunas_edu] = min_max.fit_transform(edu_2020_nordeste[colunas_edu])
edu_2020_nordeste[colunas_edu] = imputer.fit_transform(edu_2020_nordeste[colunas_edu])
edu_2020_nordeste[colunas_edu] = min_max.inverse_transform(edu_2020_nordeste[colunas_edu]).round()


# Calculando o número de escolas municipais em um Município:
edu_2020_nordeste["Total_escolas_publicas_mun_est"] = edu_2020_nordeste['Total de escolas estaduais'] \
    + edu_2020_nordeste['Total de escolas municipais']

df_edu_2020 = edu_2020_nordeste[["COD Municipio", "Total_escolas_publicas_mun_est"]]
df_edu_2020.set_index("COD Municipio", inplace=True)


# Calculando o número de matrículas por população:
total_matriculas =  edu_2020_nordeste[["COD Municipio", "Total de matrículas"]]
total_matriculas.set_index("COD Municipio", inplace=True)
df_edu_2020 = df_edu_2020.merge(total_matriculas, how="inner", left_index=True, right_index=True)


# Importando dados do Censo 2020 para o feature Engineering:
censo_raw_2020 = pd.read_excel(r"..\Dados\Censo\estimativa_dou_2020.xls", header=1, sheet_name="Municípios", dtype={"COD. MUNIC": str})
censo_2020 = censo_raw_2020.copy()
censo_2020_ne = censo_2020[censo_2020["COD. UF"].isin(codigos_ne)]
censo_2020_ne["COD. UF"] = censo_2020_ne["COD. UF"].astype(int).astype(str)

censo_2020_ne["CODIGO_MUNIC"] = censo_2020_ne["COD. UF"] +  censo_2020_ne["COD. MUNIC"]
censo_2020_ne.drop(columns=["COD. UF", "COD. MUNIC", "NOME DO MUNICÍPIO", "UF"], inplace=True)
censo_2020_ne.set_index("CODIGO_MUNIC", inplace=True)

resultado = censo_2020_ne["POPULAÇÃO ESTIMADA"].str.split("(").to_frame()
index = resultado.dropna().index
valores = [i[0] for i in resultado.dropna()["POPULAÇÃO ESTIMADA"]]

valores_serie = pd.Series(valores, index=index).str.strip()
censo_2020_ne["POPULAÇÃO ESTIMADA"].loc[index] = valores_serie.values
censo_2020_ne["POPULAÇÃO ESTIMADA"] = censo_2020_ne["POPULAÇÃO ESTIMADA"].astype(int)

# Juntando o dataset de educação com a população:
df_edu_2020 = df_edu_2020.merge(censo_2020_ne, how="inner", left_index=True, right_index=True)
df_edu_2020.index.name = "COD_MUNIC"
df_edu_2020 = df_edu_2020.reset_index()

# Feature engineering:
df_edu_2020["TOT_ESC_POR_POP"] = df_edu_2020["Total_escolas_publicas_mun_est"]/ df_edu_2020["POPULAÇÃO ESTIMADA"]
df_edu_2020["TOT_MAT_POR_POP"] = df_edu_2020["Total de matrículas"]/ df_edu_2020["POPULAÇÃO ESTIMADA"]

df_edu_2020.drop(columns=["POPULAÇÃO ESTIMADA","Total_escolas_publicas_mun_est",
                          "Total de matrículas"], inplace=True)

df_edu_2020.rename(columns={'COD_MUNIC':'IBGE7'}, inplace = True)

# Salvando a View de educação:
df_edu_2020.to_csv(r"..\Dados\Views\dados_edu_2020.csv")

df_edu_2020

,IBGE7,TOT_ESC_POR_POP,TOT_MAT_POR_POP
0,2507507,0.000354,0.207136
1,2207702,0.000704,0.249866
2,2412005,0.000627,0.238338
3,2302800,0.001139,0.270934
4,2110104,0.002872,0.343930
...,...,...,...
1789,2917003,0.001661,0.315594
1790,2103505,0.001404,0.427818
1791,2402006,0.000673,0.227836
1792,2916401,0.000586,0.230393


## 2.9) Dados de produção agrícola municipal

In [10]:
def limpar_nome(nome):
    nome_sem_acentos = unidecode.unidecode(nome)  # Remove acentos
    nome_sem_codigo_estado = re.sub(r'\(\w+\)', '', nome_sem_acentos)  # Remove a sequência "(XX)"
    return nome_sem_codigo_estado.upper()  # Converte para maiúsculas

def limpar_dataframe(df, valor):
    municipios = df['Brasil e Município'].apply(limpar_nome)
    new_df = pd.DataFrame()
    for i in range(0,3):
        new_year_df = df.iloc[:, 1 + 37 * i : 37 * (i + 1)]
        new_year_df.columns = [column.rstrip('.1').rstrip('.2') for column in new_year_df.columns]
        new_year_df.insert(0, 'MUNICIPIO', list(municipios))
        new_df = pd.concat([new_df, new_year_df], ignore_index = True)
    new_df = new_df.replace(['-', '...'], 0)
    new_df = new_df.fillna(0)
    new_df = new_df.melt(id_vars=['MUNICIPIO'], var_name='PRODUTO', value_name=valor)
    new_df[valor] = new_df[valor].astype('int64')
    new_df = new_df[['MUNICIPIO', 'PRODUTO', valor]].groupby(['MUNICIPIO', 'PRODUTO']).mean().reset_index()
    new_df['PRODUTO'] = new_df['PRODUTO'].apply(limpar_nome)
    return new_df

area_colhida = pd.read_csv("../Dados/PAM/pam_area_colhida.csv",sep=';')
area_plantada = pd.read_csv("../Dados/PAM/pam_area_plantada.csv",sep=';')
rend_medio = pd.read_csv("../Dados/PAM/pam_rendimento_medio.csv",sep=';')
valor_producao = pd.read_csv("../Dados/PAM/pam_valor_producao.csv",sep=';')

area_colhida = limpar_dataframe(area_colhida, 'AREA_COLHIDA')
area_plantada = limpar_dataframe(area_plantada, 'AREA_PLANTADA')
rend_medio = limpar_dataframe(rend_medio, 'REND_MEDIO')
valor_producao = limpar_dataframe(valor_producao, 'VALOR_PROD')

agro = area_plantada.merge(area_colhida, on = ['MUNICIPIO', 'PRODUTO'], how = 'inner')
agro = agro.merge(rend_medio, on = ['MUNICIPIO', 'PRODUTO'], how = 'inner')
agro = agro.merge(valor_producao, on = ['MUNICIPIO', 'PRODUTO'], how = 'inner')
agro['MUNICIPIO'] = agro['MUNICIPIO'].str.strip()
agro= agro[(agro['AREA_PLANTADA']>0)&(agro['AREA_COLHIDA']>0)&(agro['REND_MEDIO']>0)&(agro['VALOR_PROD']>0)]
agro.rename(columns={'MUNICIPIO':'NOME'}, inplace = True)
agro = agro[agro['NOME']!='BRASIL']
agro.to_csv("../Dados/Tabela_final/dados_producao_agricola.csv")
agro

,NOME,PRODUTO,AREA_PLANTADA,AREA_COLHIDA,REND_MEDIO,VALOR_PROD
2,ABAIARA,ALGODAO HERBACEO (EM CAROCO),26.666667,26.666667,2796.333333,214.666667
3,ABAIARA,AMENDOIM (EM CASCA),15.000000,15.000000,978.000000,60.666667
4,ABAIARA,BANANA,26.666667,26.666667,18363.333333,786.333333
12,ABAIARA,CASTANHA DE CAJU,7.000000,7.000000,333.666667,7.333333
14,ABAIARA,FAVA (EM GRAO),37.666667,37.666667,450.000000,86.666667
...,...,...,...,...,...,...
51870,XIQUE-XIQUE,TOMATE,1.000000,1.000000,9333.333333,24.666667
51891,ZABELE,FEIJAO (EM GRAO),193.333333,95.000000,219.000000,87.666667
51898,ZABELE,MANGA,4.000000,4.000000,6666.666667,31.333333
51901,ZABELE,MILHO (EM GRAO),193.333333,95.000000,429.666667,52.333333


## 2.10) Dados de tipos de solos

In [11]:
mun_ser = pd.read_csv('../Dados/Views/municipios_sertao.csv').drop(columns='Unnamed: 0')
solos = pd.read_csv('../Dados/Views/brasil_solos_5m_20201104.csv',sep=',')

solos['geometry'] = solos['the_geom'].apply(wkt.loads)
solos = solos[['LEG_SINOT', 'geometry']]
solos = gpd.GeoDataFrame(solos, geometry='geometry')
solos.crs = 'EPSG:4674'

lim_muns = pd.read_json('../Dados/Views/municipios-poligonos.json')
lim_muns = lim_muns[lim_muns['municipioCodigo'].isin(mun_ser['IBGE7'].unique())]
lim_muns['poligono'] = [str(polygon) for polygon in lim_muns['poligono']]
lim_muns['geometry'] = lim_muns['poligono'].apply(lambda x: shape(eval(x)))
lim_muns = lim_muns[['municipioCodigo', 'geometry']]
lim_muns.columns = ['IBGE7', 'geometry']
lim_muns = gpd.GeoDataFrame(lim_muns, geometry='geometry')
lim_muns.crs = 'EPSG:4674'

intersecao_gdf = gpd.overlay(solos, lim_muns, how='intersection')
intersecao_gdf['AreaSoloHectares'] = intersecao_gdf['geometry'].area / 10000
merged_data = lim_muns.merge(intersecao_gdf.groupby(['IBGE7', 'LEG_SINOT'])['AreaSoloHectares'].sum().reset_index(), on='IBGE7', how='left')
merged_data.rename(columns={'LEG_SINOT': 'SOLO', 'AreaSoloHectares': 'AREA_TOTAL'}, inplace=True)
tabela_final = merged_data[['IBGE7', 'SOLO', 'AREA_TOTAL']]
tabela_final.to_csv('../Dados/Views/solos_municipios.csv')
tabela_final

,IBGE7,SOLO,AREA_TOTAL
0,2207959,LAd - Latossolos Amarelos Distroficos,5.171840e-06
1,2207959,PVAe - Argissolos Vermelho-Amarelos Eutroficos,3.887957e-07
2,2207959,RQo - Neossolos Quartzarenicos Orticos,1.970030e-06
3,2207934,PVAe - Argissolos Vermelho-Amarelos Eutroficos,1.849599e-06
4,2207934,RQo - Neossolos Quartzarenicos Orticos,5.210986e-06
...,...,...,...
3396,3161106,CXbd - Cambissolos Haplicos Tb Distroficos,2.420436e-07
3397,3161106,CXve - Cambissolos Haplicos Ta Eutroficos,5.223050e-06
3398,3161106,PVe - Argissolos Vermelhos Eutroficos,3.197621e-06
3399,3161106,RQo - Neossolos Quartzarenicos Orticos,1.349726e-05


## 2.11) Tabela final de dados socioeconômicos dos municípios

In [12]:
mun_ser = pd.read_csv('../Dados/Views/municipios_sertao.csv').drop(columns = 'Unnamed: 0')
reg_adm_mun = pd.read_csv('../Dados/Views/registros_administrativos.csv').drop(columns = 'Unnamed: 0')
met = pd.read_csv('../Dados/Views/dados_meteorologicos.csv').drop(columns = 'Unnamed: 0')
rec_h = pd.read_csv('../Dados/Views/recursos_hidricos.csv').drop(columns = 'Unnamed: 0')
censo = pd.read_csv('../Dados/Views/censo.csv').drop(columns = 'Unnamed: 0')
ql_agua = pd.read_csv('../Dados/Views/qualidade_da_agua.csv').drop(columns = 'Unnamed: 0')
trans = pd.read_csv('../Dados/Views/custo_de_transporte.csv').drop(columns = 'Unnamed: 0')
edu = pd.read_csv(r"..\Dados\Views\dados_edu_2020.csv").drop(columns = 'Unnamed: 0')

tabela_final = mun_ser.copy()
tabela_final = tabela_final.merge(reg_adm_mun, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(met, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(rec_h, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(censo, on= 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(ql_agua, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(trans, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(edu, on= 'IBGE7', how = 'left')

num_metrics = [column for column in tabela_final.columns if column not in ['IBGE7', 'UF', 'NOME', 'LATITUDE', 'LONGITUDE']]

# Imputation step:
#imputer = IterativeImputer(random_state=42)
imputer = KNNImputer(n_neighbors=5, weights="uniform")
min_max = MinMaxScaler()

tabela_final[num_metrics] = min_max.fit_transform(tabela_final[num_metrics])
tabela_final[num_metrics] = imputer.fit_transform(tabela_final[num_metrics])
tabela_final[num_metrics] = min_max.inverse_transform(tabela_final[num_metrics])

tabela_final.to_csv('../Dados/Tabela_final/tabela_final.csv', index=False)

## 2.12) Distância para a capital mais próxima

In [14]:
def limpar_nome(nome):
    nome_sem_acentos = unidecode.unidecode(nome)  
    nome_sem_codigo_estado = re.sub(r'\(\w+\)', '', nome_sem_acentos)  
    return nome_sem_codigo_estado.upper()

def calcular_distancia_minima(row):
    municipio = (row['LATITUDE'], row['LONGITUDE'])
    distancias = []
    for _, capital in mun_capitais.iterrows():
        capital_coords = (capital['LATITUDE'], capital['LONGITUDE'])
        dist = great_circle(municipio, capital_coords).kilometers
        distancias.append(dist)
    return min(distancias)

codigos_ibge_capitais = [
    2800308, 1501402, 3106200, 1400100, 5300108, 5002704, 5103403, 4314902,
    4205407, 2304400, 5208707, 2507507, 1600303, 2704302, 1302603, 2408102,
    1100205, 2611606, 1200401, 3304557, 2927408, 2111300, 3550308, 2211001,
    3205309, 1721000
]

mun_capitais = pd.read_csv(r'../Dados/Municipios/municipios.csv')
mun_capitais['nome'] = mun_capitais['nome'].apply(limpar_nome)
mun_capitais = mun_capitais[mun_capitais['codigo_ibge'].isin(codigos_ibge_capitais)].drop(columns='codigo_uf')
mun_capitais.columns = ['IBGE7', 'NOME', 'LATITUDE', 'LONGITUDE']

mun_ser = pd.read_csv('../Dados/Views/municipios_sertao.csv')
mun_ser['DIST_CAPITAL'] = mun_ser.apply(calcular_distancia_minima, axis=1)
mun_ser = mun_ser[['IBGE7', 'DIST_CAPITAL']]
mun_ser.to_csv('../Dados/Views/distancia_capitais.csv', index=False)
mun_ser

,IBGE7,DIST_CAPITAL
0,2700300,102.025981
1,2700706,137.500525
2,2700904,122.950205
3,2701209,140.854338
4,2701605,207.272744
...,...,...
1256,2806008,57.971219
1257,2807006,67.763439
1258,2807105,83.224418
1259,2807303,80.736011
